# Baixando o Dataset

O código abaixo baixa o arquivo `Iris.csv`

In [ ]:
!gdown --id '1d3NbjXro_BfnYpFm66ETBfe7ubAZPAoL'

Downloading...
From: https://drive.google.com/uc?id=1d3NbjXro_BfnYpFm66ETBfe7ubAZPAoL
To: /content/Iris.csv
100% 5.11k/5.11k [00:00<00:00, 4.46MB/s]


# Importação do PyTorch

A seguir importamos a biblioteca PyTorch

In [ ]:
#Aqui importamos o módulo do PyTorch

import torch

# Em seguida ajustamos para suprimir a
# notação científica na hora de imprimir
# os tensores

torch.set_printoptions(precision=2,sci_mode=False)

# Leitura dos dados

Nas linhas seguintes fazemos a leitura dos dados do arquivo.

In [ ]:
# Abre o arquivo e faz uma leitura
# das suas linhas

f = open('Iris.csv', 'r')
lines = f.readlines()

In [ ]:
# 4 entradas
X = torch.zeros(len(lines)-1,4)

# 3 saídas desejadas (one-hot)
Y = torch.zeros(len(lines)-1,3)

# Teremos 3 categorias. O vetor abaixo lista as strings
# que representam as categorias possíveis
cats = ['Iris-setosa','Iris-versicolor','Iris-virginica']

# Para cada linha do arquivo, exceto
# a primeira linha que é o cabeçalho
for i, line in enumerate(lines[1:]):

  # Aqui decodificamos a linha para transformar
  # de binário para caracteres ascii, e descartamos
  # o último caractere que representa uma nova linha
  s = line[:-1]

  # Aqui separamos os dados por vírgulas,
  # descartando o primeiro valor que é o id
  # pois usaremos i do laço como id.
  _,sl,sw,pl,pw,sp = s.split(',')

  # Transformamos as strings que representam
  # as dimensões de sépala e pétala para ponto
  # flutuante.
  sl = float(sl)
  sw = float(sw)
  pl = float(pl)
  pw = float(pw)
  
  # Aqui populamos as matrizes X e Y com os dados
  # coletados.
  X[i,:] = torch.tensor([sl,sw,pl,pw])
  Y[i,:] = torch.tensor([1 if sp == cat else 0 for cat in cats])

# Embaralhamento e Separação de Dados de Validação

A seguir embaralhamos os pares de entradas e respectivas saídas desejadas, separando uma parte das amostras para validação do resultado do treinamento da rede neural.

In [ ]:
import random

# Aqui criamos uma lista de índices
# embaralhados
indexes = list(range(150))
random.shuffle(indexes)

# Essa variável T indica quantas amostras
# serão usadas para treinamento. As demais
# serão usadas para validação
T = 140

# Aqui preparamos as matrizes dos pares
# de dados de treinamento e validação.
Xt = torch.zeros(T,4)
Yt = torch.zeros(T,3)
Xv = torch.zeros(150-T,4)
Yv = torch.zeros(150-T,3)

# Aqui preenchemos as matrizes com os
# respectivos valores
for i in range(0,T):
  Xt[i,:] = X[indexes[i],:]
  Yt[i,:] = Y[indexes[i],:]
for i in range(0,150-T):
  Xv[i,:] = X[indexes[T+i],:]
  Yv[i,:] = Y[indexes[T+i],:]

# Módulos auxiliares

Esses submódulos do PyTorch facilitam a construção de redes neurais.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Definição da Arquitetura

No código a seguir criamos a rede neural como uma classe que deriva de `nn.Module`.

In [ ]:
# Essa é a classe da rede neural

class Perceptron(nn.Module):

  def __init__(self):

    # Chamamos o construtor da classe
    # mãe (nn.Module)
    
    super(Perceptron, self).__init__()

    # Criamos os objetos que implementam
    # a parte referente à combinação linear
    # (pesos sinapticos e biases)

    self.c1 = nn.Linear(4, 8)
    self.c2 = nn.Linear(8, 3)

  def forward(self, x):

    # Aqui calculamos a parte
    # linear da primeira
    # camada

    s1 = self.c1(x)

    # Aplicamos a função de
    # ativação sigmoide

    z1 = torch.sigmoid(s1)

    # E calculamos a parte
    # linear da segunda e
    # última camada.
    s2 = self.c2(z1)

    # Retornamos assim mesmo,
    # sem ativação, já que a
    # função softmax é implementada
    # diretamente na hora de
    # calcular o custo
    
    return s2

In [ ]:
# Aqui instanciamos o objeto

p = Perceptron()
print(p)

Perceptron(
  (c1): Linear(in_features=4, out_features=8, bias=True)
  (c2): Linear(in_features=8, out_features=3, bias=True)
)


In [ ]:
# Esse método herdado da classe mãe
# mostra todos os parâmetros, que
# nesse caso são os pesos e biases
# de todas as camadas.

list(p.parameters())

[Parameter containing:
 tensor([[ 0.15, -0.15,  0.12, -0.13],
         [-0.01,  0.27, -0.06, -0.02],
         [ 0.11, -0.22,  0.48,  0.43],
         [ 0.18, -0.14,  0.27,  0.05],
         [-0.24,  0.47,  0.49,  0.13],
         [-0.26, -0.06,  0.46,  0.08],
         [ 0.20,  0.19, -0.48,  0.24],
         [-0.27,  0.09, -0.36,  0.09]], requires_grad=True),
 Parameter containing:
 tensor([ 0.10,  0.27, -0.48, -0.01,  0.45,  0.14,  0.01, -0.23],
        requires_grad=True),
 Parameter containing:
 tensor([[-0.29,  0.02, -0.11,  0.31, -0.01, -0.18, -0.02, -0.30],
         [-0.15, -0.28, -0.02,  0.12,  0.30, -0.19,  0.08,  0.25],
         [-0.02, -0.15,  0.31,  0.12,  0.03, -0.27,  0.20, -0.28]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.26,  0.29, -0.17], requires_grad=True)]

# Função de Perda e Treinamento

No código a seguir mostramos como pode ser aplicada uma função de perda e realizado o treinamento da rede neural.

In [ ]:
# Aqui criamos uma entrada arbitrária

x = torch.tensor([[1.0,2.0,3.0,4.0]])

# Calculamos a saída da rede neural
# mesmo que ainda usando pesos e biases
# aleatórios

y = p(x)

# E criamos uma saída desejada (arbitrária,
# só para demonstração)
y_hat = torch.tensor([[0.0, 1.0, 0.0]])

# Essa linha de código cria o objeto
# que implementa a função de perda do
# tipo Entropia Cruzada
loss = nn.CrossEntropyLoss()

# Aqui usamos o objeto criado para calcular
# a função de perda aplicada à saída calculada
# e saída desejada.

e = loss(y, y_hat.argmax(dim=1))
print(e)

tensor(0.86, grad_fn=<NllLossBackward>)


In [ ]:
# Esse erro acima pode ser propagado
# para os parâmetros, calculando as
# derivadas parciais do erro para
# cada respectivo parâmetro (gradiente
# do erro)

# Aqui zeramos o gradiente (zeramos
# as derivadas parciais do erro em
# relação a cada um dos parâmetros
# da rede neural)

p.zero_grad()

# Em seguida propagammos o gradiente
# do erro de trás para frente, usando
# a regra da cadeia (isso calcula as
# derivadas parciais do erro para cada
# um dos parâmetros da rede neural)

e.backward()

# No laço abaixo aplicamos uma pequena
# correção de um passo de tamanho 0.1
# no sentido contrário ao do gradiente
# de cada parâmetro (descida do gradiente)

for param in p.parameters():
  param.data -= 0.1*param.grad

In [ ]:
# O código a seguir combina todas essas
# etapas acima, fazendo efetivamente
# o treinamento da rede neural para
# todos os dados de treinamento que
# já tínhamos separado inicialmente

# Laço para 1001 épocas

for i in range(1001):

  # Para cada par de entrada
  # e respectiva saída desejada
  for x, y_hat in zip(Xt,Yt):

    # Corrigimos o formato do
    # par para que cada um seja
    # um vetor linha (que é como
    # o PyTorch espera)

    x = x.view(1,4)
    y_hat = y.view(1,3)

    # Zeramos os gradientes dos
    # parâmetros para receberem
    # os resultados novos
    p.zero_grad()

    # Computamos a saída calculada
    # (caminho direto da rede neural)

    y = p(x)

    # Calculamos o erro entre a saída
    # calculada e a desejada (lembrando
    # que esse erro permite calcular
    # o gradiente)
    e = loss(y, y_hat.argmax(dim=1))

    # Chamamos a função que computa
    # o gradiente da função de erro
    # para todos parâmetros da rede
    # neural

    e.backward()

    # Aqui aplicamos uma correção dos
    # parâmetros um passo de 0.1 na
    # direção oposta ao gradiente do
    # erro (subtraino 0.1 vezes a
    # derivada parcial do erro em
    # relação a cada respectivo
    # parâmetro)

    for param in p.parameters():
      param.data -= 0.1*param.grad.data
  
  # Imprimimos uma vez para
  # cada 100 amostras
  if not (i % 100):
    print(float(e))

0.009103813208639622
7.60526381782256e-05
3.814624506048858e-05
2.5510462364763953e-05
1.9073304429184645e-05
1.537788011773955e-05
1.2755313036905136e-05
1.0847986231965479e-05
9.536697689327411e-06
8.4638240878121e-06
7.629365427419543e-06


In [ ]:
# O código dessa célula demonstra
# como usar um otimizador, ao invés
# de simplesmente subtrair passo vezes
# gradiente.

# Importamos o módulo que implementa
# os otimizadores

import torch.optim as optim

# Escolhemos aqui o stochastic gradient descent
# que é um dos otimizadores mais simples
# que existem.

optimizer = optim.SGD(p.parameters(), lr=0.1)

# Esse laço aplica o otimizador acima
# para todo dataset de treinamento,
# repetindo o processo 10001 vezes.

for i in range(10001):

  # Zeramos os gradientes

  optimizer.zero_grad()

  # Calculamos as saídas da
  # rede neural de uma vez,
  # em lote, para todas entradas
  # de treinamento.

  Y = p(Xt)

  # Calculamos o erro, em lote
  e = loss(Y, Yt.argmax(dim=1))

  # Fazemos a propagação reversa
  # dos gradientes desse erro do
  # lote

  e.backward()

  # Aplicamos um passo do otimizador
  optimizer.step()

  # Aqui imprimimos uma vez
  # a cada mil épocas
  if not (i % 1000):
    print(float(e))

1.1354684829711914
0.14748774468898773
0.08476033061742783
0.07048087567090988
0.06433117389678955
0.06082615256309509
0.05849975347518921
0.05680329352617264
0.05548500269651413
0.05441270396113396
0.05350981652736664


# Rodando otimização na GPU

Para rodar na GPU basta mover todos os dados para a GPU e rodar novamente a célula acima para que o treinamento aconteça dessa vez na GPU.

In [ ]:
# Cria o objeto que representa
# a GPU usando CUDA (NVidia)
gpu = torch.device("cuda:0")

# Move rede neural e dados de
# treinamento e validação, todos
# para a GPU

p.to(gpu)
Xt = Xt.to(gpu)
Yt = Yt.to(gpu)
Xv = Xv.to(gpu)
Yv = Yv.to(gpu)

# Verificação dos Resultados

Esse código abaixo mostra o resultado da rede neural aplicada aos dados de validação, comparando saída calculada e desejada.

In [ ]:
Y = F.softmax(p(Xv), dim=1)
for y, y_hat in zip(Y, Yv):
  print(y.data, y_hat.data)

tensor([    0.00,     0.00,     1.00]) tensor([0., 0., 1.])
tensor([    1.00,     0.00,     0.00]) tensor([1., 0., 0.])
tensor([    0.00,     1.00,     0.00]) tensor([0., 1., 0.])
tensor([    0.00,     0.01,     0.99]) tensor([0., 0., 1.])
tensor([    1.00,     0.00,     0.00]) tensor([1., 0., 0.])
tensor([    0.00,     0.00,     1.00]) tensor([0., 0., 1.])
tensor([    1.00,     0.00,     0.00]) tensor([1., 0., 0.])
tensor([    0.00,     0.15,     0.85]) tensor([0., 0., 1.])
tensor([    0.00,     0.99,     0.01]) tensor([0., 1., 0.])
tensor([    0.00,     0.99,     0.01]) tensor([0., 1., 0.])
